In [3]:
spark.sql("show databases").show()


In [4]:
# schema = "event_time timestamp, event_type string, product_id long, category_id long, category_code string, brand string, price float, user_id long, user_session string"

df = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv("data/data_nov.csv")

df.printSchema()

df \
.write.mode("overwrite") \
.format("parquet") \
.save("data/data_nov.parquet")

In [5]:
df.show()

In [8]:
spark.read.parquet("data/data_nov.parquet").coalesce(1).write.mode("overwrite").parquet("data/data_nov_coalesce.parquet")

In [10]:
events_df = spark.read.parquet("data/data_nov_coalesce.parquet")

events_df.printSchema()
events_df.show(100)

In [11]:
events_df.select(f.min("event_time"), f.max("event_time")).show(truncate=False)

In [12]:
events_df.where("brand = 'intel'").show()

In [13]:
print("Hello " + z.textbox("name"))

In [14]:
spark.sql("create database src_data")

events_df.write.saveAsTable("src_data.events")

In [17]:
import pyspark.sql.functions as f


spark.sql("create database market")

spark.table("src_data.events") \
.withColumn("date", f.to_date("event_time")) \
.write.partitionBy("date").saveAsTable("market.events")

In [20]:
import pyspark.sql.functions as f

events_df \
    .groupBy("event_type") \
    .agg(
        f.count("*").alias("count")
        ) \
    .orderBy(f.col("count").desc()) \
    .show(truncate=False)
